In [ ]:
import pandas as pds
import numpy as np

In [ ]:
train = pds.read_csv('titanic/train.csv')
test = pds.read_csv('titanic/test.csv')

In [ ]:
train_test_dataset = [train,test]

In [ ]:
for data in train_test_dataset :
    data['Title'] = data['Name'].str.extract('([a-zA-Z]+)\.')
    data['Title'] = data['Title'].str.lower()

    data['Title'] = np.where(data['Title'] == 'mr', 0, 
                             np.where(data['Title'] == 'miss', 1, 
                                      np.where(data['Title'] == 'mrs', 2, 3)))

    data['Age'].fillna(data.groupby('Title')['Age'].transform('median'), inplace = True)

    data['Age'] = np.where(data['Age'] <= 16, 0, 
                           np.where(data['Age'] <= 26, 1, 
                                    np.where(data['Age'] <= 36, 2, 
                                             np.where(data['Age'] <= 62, 3, 4))))
    
    data['Embarked'].fillna('S', inplace=True)
    
    data['Embarked'] = np.where(data['Embarked'] == 'S' , 0, 
                                np.where(data['Embarked'] == 'C', 1, 2))
    
    data['Cabin'] = data['Cabin'].str[0]
    
    cabin_mapping = {'A' : 0 , 'B' : 0.4, 'C' : 0.8, 'D' : 1.2, 'E' : 1.6, 'F' : 2.0, 'G' : 2.4, 'T' : 2.8}
        
    data['Cabin'] = data['Cabin'].map(cabin_mapping)
    
    data['Cabin'].fillna(data.groupby('Pclass')['Cabin'].transform('median'), inplace=True)
    
    data['Sex'] =data['Sex'].map({'male' : 0,'female' : 1})
    
    data['Fare'].fillna(data.groupby('Pclass')['Fare'].transform('median'), inplace=True)
    data['Fare'] = np.where(data['Fare'] <= 17, 0, 
                            np.where(data['Fare'] <= 30, 1, 
                                     np.where(data['Fare'] <= 100, 2, 3)))
    
    
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    
    family_mapping = {1: 0.0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2.0, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4.0}
        
    data['FamilySize'] = data['FamilySize'].map(family_mapping)

In [ ]:
features_drop = ['PassengerId', 'Name', 'Ticket', 'SibSp', 'Parch',]

In [ ]:
train = train.drop(features_drop, axis = 1)

In [ ]:
test = test.drop(features_drop, axis = 1)

In [ ]:
train.head(1)

In [ ]:
test.head(1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
train

In [ ]:
inputs = train.drop('Survived', axis = 1) 
# train.drop(columns= 'Survived')

In [ ]:
outputs = train['Survived']

In [ ]:
print(inputs.shape, outputs.shape) # 아무것도 안 보이는 것은 1이 생략된 것임. (데이터 수 , 속성값)

In [ ]:
train_input, test_input, train_output, test_output = train_test_split(inputs, outputs, test_size = 0.2, stratify = outputs, random_state = 10) 
# input을 x output을 y로 자주 쓴다. 해당 순서 외우는 편이 좋음 (x_train, x_test, y_train, y_output)

In [ ]:
print(train_input.shape, train_output.shape)
print(test_input.shape, test_output.shape)

In [ ]:
train_input.isnull().sum()

In [ ]:
outputs.value_counts() # 0이 죽은 사람. 대략 6:4 정도로 가정. train이나 test나 비율을 유지하는 것이 좋다. (stratify = outputs 옵션)

In [ ]:
train_output.value_counts()

In [ ]:
test_output.value_counts()

---

In [ ]:
model1 = LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = RandomForestClassifier()
model4 = GradientBoostingClassifier()
model5 = AdaBoostClassifier()
# 괄호 꼭 붙이기. 앞서서 null값 없는지 확인해봐야 함. null값이 있으면 이하 fit이 돌아가지 않는다.

In [ ]:
model1.fit(train_input, train_output) # 문제, 정답 순으로 입력

In [ ]:
model1.score(train_input, train_output)

In [ ]:
model1.score(test_input, test_output)

In [ ]:
test_pre = model1.predict(test_input)

In [ ]:
test_input.iloc[0]

In [ ]:
test_pre[0]

In [ ]:
test_output.iloc[0]

In [ ]:
accuracy_score(test_pre, test_output) *100

In [ ]:
model= {'LogisticRegression' : LogisticRegression(), 
        'DecisionTreeClassifier' : DecisionTreeClassifier(),
        'RandomForestClassifier' : RandomForestClassifier(),
        'GradientBoostingClassifier' : GradientBoostingClassifier(),
        'AdaBoostClassifier' : AdaBoostClassifier()}

In [ ]:
for m in model :
    clf = model[m]
    clf.fit(train_input, train_output)
    train_score = clf.score(train_input, train_output) * 100
    test_score = clf.score(test_input, test_output) * 100
    
    print(f"[%26s] train:%.2f / test:%.2f" % (m, train_score, test_score))
# 분명 위에서 데이터가 좀 다른듯... 

In [ ]:
# 얘들을 데이터 프레임으로 만들고 싶으면? 

In [ ]:
model_l =[]
train_l = []
test_l = []

for m in model :
    clf = model[m]
    clf.fit(train_input, train_output)
    train_score = clf.score(train_input, train_output) * 100
    test_score = clf.score(test_input, test_output) * 100
    
    model_l.append(m)
    train_l.append(train_score)
    test_l.append(test_score)
    
    print(f"[%26s] train:%.2f / test:%.2f" % (m, train_score, test_score))
# 분명 위에서 데이터가 좀 다른듯... 

In [ ]:
model = GradientBoostingClassifier()

In [ ]:
model.fit(train_input, train_output)

In [ ]:
model

In [ ]:
import pickle

In [ ]:
with open ('gbc.pkl', 'wb') as f: #바이너리 파일로 쓰라는 뜻
    pickle.dump(model, f)

In [ ]:
with open ('gbc.pkl', 'rb') as f:
    load_model = pickle.load(f)

In [ ]:
load_model

In [ ]:
data = test_input.iloc[0].to_list()

In [ ]:
import numpy as np

In [ ]:
np_data = np.asarray(data)

In [ ]:
re_data = np_data.reshape(1,-1)

In [ ]:
re_data

In [ ]:
pred = load_model.predict(re_data)

In [ ]:
if pred[0] == 0 :
    print('Died...')
else:
    print('Survived')

----

In [ ]:
test 

In [ ]:
test.isnull().sum()

In [ ]:
sub = pds.read_csv('titanic/submission.csv')

In [ ]:
answer = sub['Survived']

In [ ]:
answer

In [ ]:
submission = load_model.predict(test)

In [ ]:
submission

In [ ]:
print(answer.shape, submission.shape)

In [ ]:
accuracy_score(answer, submission) * 100

In [ ]:
sub2 = pds.read_csv('titanic/submission2.csv')

In [ ]:
sub2

In [ ]:
sub2['Survived'] = submission

In [ ]:
sub2

In [ ]:
sub2.to_csv('result.csv', index = False)